In [7]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [8]:
from os import environ
from citrination_client import CitrinationClient
from citrination_client import *

In [9]:
from pypif import pif
from pypif.obj import *
import csv

In [10]:
import pandas as pd

# Table 1:

In [11]:
client = CitrinationClient(environ['CITRINATION_API_KEY'], 'https://citrination.com')
dataset_id = '151803'
value_query = FieldQuery(extract_as="Elastic Modulus", extract_all=True)
property_query = PropertyQuery(name=FieldQuery(filter=[Filter(equal="Elastic Modulus")]), value=value_query)
formula_query = ChemicalFieldQuery(extract_as="formula")
system_query = PifSystemQuery(chemical_formula=formula_query, properties=property_query)
dataset_query = DatasetQuery(id=[Filter(equal=dataset_id)])
data_query = DataQuery(dataset=dataset_query, system=system_query)
pif_query = PifSystemReturningQuery(size=5000, random_results=True, query=data_query)
search_result = client.search.pif_search(pif_query)

print("We found {} records".format(len(search_result.hits)))
print([x.extracted for x in search_result.hits[0:2]])

We found 381 records
[{'formula': 'SiC', 'Elastic Modulus': ['138', '166', '178', '194', '180', '146', '144', '178', '223', '204', '249', '317']}, {'formula': 'ZrO2.xY2O3', 'Elastic Modulus': ['200', '200', '200']}]


Gererating table for Elastic modulus vs Temperature

In [12]:
rows = []
pif_records = [x.system for x in search_result.hits]
for system in pif_records:
    if "x" not in system.chemical_formula and "." not in system.chemical_formula:
        cryst_value= None
        for prop in system.properties:
            if prop.name == 'Crystallinity':
                cryst_value= prop.scalars[0].value
        for prop in system.properties:
            if prop.name == "Elastic Modulus" and prop.units == "GPa":
                for cond in prop.conditions:
                    if cond.name == "Temperature":
                        if len(prop.scalars) == len(cond.scalars):
                            for prop_sca, cond_sca in zip(prop.scalars, cond.scalars):
                                row = [system.chemical_formula, prop_sca.value, cond_sca.value, 
                                       cryst_value, system.references[0].citation]
                                rows.append(row)

with open('elastic_modulus_temperature.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Formula', 'Elastic Modulus (GPa)', 'Temperature (K)', 'Crystallinity','Reference'])
    writer.writerows(rows)

Generating csv for elastic modulus vs Volume Fraction of Porosity

In [13]:
rows = []
pif_records = [x.system for x in search_result.hits]
for system in pif_records:
    if "x" not in system.chemical_formula and "." not in system.chemical_formula:
        cryst_value= None
        for prop in system.properties:
            if prop.name == 'Crystallinity':
                cryst_value= prop.scalars[0].value
        for prop in system.properties:
            if prop.name == "Elastic Modulus" and prop.units == "GPa":
                for cond in prop.conditions:
                    if cond.name == "Volume Fraction of Porosity":
                        if len(prop.scalars) == len(cond.scalars):
                            for prop_sca, cond_sca in zip(prop.scalars, cond.scalars):
                                if '_units' in cond.__dict__.keys():
                                    if cond.units == '%':
                                        cond_sca.value = float(cond_sca.value)/100.
                                row = [system.chemical_formula, prop_sca.value, cond_sca.value, 
                                       cryst_value, system.references[0].citation]
                                rows.append(row)

with open('elastic_modulus_porosity.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Formula', 'Elastic Modulus (GPa)', 'Volume Fraction of Porosity', 'Crystallinity','Reference'])
    writer.writerows(rows)

# Table 2:

In [14]:
from matminer.utils.conversions import str_to_composition
from matminer.featurizers import composition
from matminer.featurizers import structure

Elastic Modulus vs. Temperature

In [15]:
# select featurizers

# elemental property
ep_feat = composition.ElementProperty.from_preset(preset_name="magpie")

# atomic packing efficiency
ape_feat= composition.AtomicPackingEfficiency()

# density features
d_feat= structure.DensityFeatures()

In [26]:
dat_t= pd.read_csv('elastic_modulus_temperature.csv',header= 0)
dat_t= dat_t.drop_duplicates(['Formula','Temperature (K)','Crystallinity'])

In [27]:
dat_t['composition']= dat_t['Formula'].transform(str_to_composition)
dat_t= ep_feat.featurize_dataframe(dat_t,col_id='composition',ignore_errors=True)
dat_t= ape_feat.featurize_dataframe(dat_t,col_id='composition',ignore_errors=True)
#dat_t= d_feat.featurize_dataframe(dat_t,col_id='structure',ignore_errors=True)

In [28]:
dat_t.to_pickle('elastic_modulus_temperature_with_features.pkl')

Elastic Modulus vs. Porosity

In [35]:
dat_p= pd.read_csv('elastic_modulus_porosity.csv',header= 0)
dat_p= dat_p.drop_duplicates(['Formula', 'Volume Fraction of Porosity','Crystallinity'])

In [37]:
dat_p['composition']= dat_p['Formula'].transform(str_to_composition)
dat_p= ep_feat.featurize_dataframe(dat_p,col_id='composition',ignore_errors=True)
dat_p= ape_feat.featurize_dataframe(dat_p,col_id='composition',ignore_errors=True)
#dat_p= d_feat.featurize_dataframe(dat_p,col_id='structure',ignore_errors=True)

In [38]:
dat_p.to_pickle('elastic_modulus_porosity_with_features.pkl')